In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip3 install pytesseract
#!pip3 install tesseract
#!pip3 install pillow

# Then install Tesseract using apt-get
!apt-get install tesseract-ocr
!pip install pytesseract Pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,006 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 120903 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!pip install gTTS
!pip install --upgrade gTTS

In [ ]:
import io
import json
import cv2
import numpy as np
import requests
from PIL import Image
#import pytesseract
from gtts import gTTS
import IPython
from IPython.display import Audio
import IPython.display as ipd
import os
import time




In [ ]:

#from art import tprint
#import pytesseract

def play_audio(text):

    tts = gTTS(text)
    tts.save("/content/drive/MyDrive/Thesis_practice/Code 2/outputaudio.mp3")  # Save the audio to a file (replace "/content/output.mp3" with your desired file path)

def apply_yolo_object_detection(image_to_process):
    """
    Recognition and determination of the coordinates of objects on the image
    :param image_to_process: original image
    :return: image with marked objects and captions to them
    """

    height, width, _ = image_to_process.shape
    blob = cv2.dnn.blobFromImage(image_to_process, 1 / 255, (608, 608),
                                 (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)
    class_indexes, class_scores, boxes = ([] for i in range(3))
    objects_count = 0
    detected_objects = []
    true_positives = []

    # Starting a search for objects in an image
    for out in outs:
        for obj in out:
            scores = obj[5:]
            class_index = np.argmax(scores)
            class_score = scores[class_index]
            if class_score > 0:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                obj_width = int(obj[2] * width)
                obj_height = int(obj[3] * height)
                box = [center_x - obj_width // 2, center_y - obj_height // 2,
                       obj_width, obj_height]
                #boxes.append(box)
                class_name = classes[class_index]
                detected_objects.append((class_name,box))
                yolo_box = [center_x - obj_width // 2, center_y - obj_height // 2, obj_width, obj_height]



                #class_indexes.append(class_index)
                #class_scores.append(float(class_score))

    return detected_objects

    # Selection
    # chosen_boxes = cv2.dnn.NMSBoxes(boxes, class_scores, 0.0, 0.4)
    # for box_index in chosen_boxes:
    #     box_index = box_index
    #     box = boxes[box_index]
    #     class_index = class_indexes[box_index]

    #     # For debugging, we draw objects included in the desired classes
    #     if classes[class_index] in classes_to_look_for:
    #         objects_count += 1
    #         image_to_process = draw_object_bounding_box(image_to_process,
    #                                                     class_index, box)

    # final_image = draw_object_count(image_to_process, objects_count)
    # return final_image


def draw_object_bounding_box(image_to_process, index, box):
    """
    Drawing object borders with captions
    :param image_to_process: original image
    :param index: index of object class defined with YOLO
    :param box: coordinates of the area around the object
    :return: image with marked objects
    """

    x, y, w, h = box
    start = (x, y)
    end = (x + w, y + h)
    color = (0, 255, 0)
    width = 2
    final_image = cv2.rectangle(image_to_process, start, end, color, width)

    start = (x, y - 10)
    font_size = 1
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 2
    text = classes[index]
    final_image = cv2.putText(final_image, text, start, font,
                              font_size, color, width, cv2.LINE_AA)

    return final_image


def draw_object_count(image_to_process, objects_count):
    """
    Signature of the number of found objects in the image
    :param image_to_process: original image
    :param objects_count: the number of objects of the desired class
    :return: image with labeled number of found objects
    """

    start = (10, 120)
    font_size = 1.5
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 3
    text = "Objects found: " + str(objects_count)

    # Text output with a stroke
    # (so that it can be seen in different lighting conditions of the picture)
    white_color = (255, 255, 255)
    black_outline_color = (0, 0, 0)
    final_image = cv2.putText(image_to_process, text, start, font, font_size,
                              black_outline_color, width * 3, cv2.LINE_AA)
    final_image = cv2.putText(final_image, text, start, font, font_size,
                              white_color, width, cv2.LINE_AA)

    return final_image

from google.colab.patches import cv2_imshow

def start_video_object_detection(video: str):
    """
    Захват и анализ видео в режиме реального времени
    """

    # Define video output parameters
    while True:
        try:
            # Capturing a picture from a video
            video_camera_capture = cv2.VideoCapture(video)
            class_counts = {class_name: 0 for class_name in classes}


            while video_camera_capture.isOpened():
                ret, frame = video_camera_capture.read()
                if not ret:
                    break
                detected_objects = apply_yolo_object_detection(frame)

                # Application of object recognition methods on a video frame from YOLO
                #detected_objects = apply_yolo_object_detection(frame)
                #dire = r'/content/drive/MyDrive/Thesis_practice/output_image.jpg'  # You can set the desired output file name and extension
                #cv2.imwrite(dire, frame)    # Assuming you're using OpenCV to work with images
                #image = Image.open(dire)

                #bbox_coords = (x, y, width, height)

                #roi = image.crop(bbox_coords)
                #try:
                   #text = pytesseract.image_to_string(image, lang='eng')
                   #print(text)
                #except Exception as e:
                   #print("An error occurred:", str(e))
                #filename=r'/content/drive/MyDrive/Thesis_practice/Code 2/output.txt'
                #file1 = open(filename+".txt","w")
                #file1.write(text)
                #file1.close()
                #print("Done...")

                #im = Image.open(dire)
                #text = pytesseract.image_to_string(im, lang=language)
                #filename=input("Type output file name:")
                #file1 = open(filename+"-"+language+".txt","w")
                #file1.write(text)
                #file1.close()
                #print("Done...")

                # Displaying the processed image on the screen with a reduced window size
                #recognizer = cv2.face.createLBPHFaceRecognizer()

                #frame = cv2.resize(frame, (1920 // 2, 1080 // 2))
                #cv2.cv2_imshow("Video Capture", frame)
                total_objects = 0
                for class_name, box in detected_objects:
                   x, y, w, h = box
                   start = (x, y)
                   end = (x + w, y + h)
                   color = (0, 255, 0)
                   width = 2
                   frame = cv2.rectangle(frame, start, end, color, width)

                   start = (x, y - 10)
                   font_size = 1
                   font = cv2.FONT_HERSHEY_SIMPLEX
                   width = 2
                   text = class_name
                   frame = cv2.putText(frame, text, start, font, font_size, color, width, cv2.LINE_AA)

                   class_counts[class_name] += 1
                   total_objects += 1


               # Print the detected objects
                text_to_speek = ""
                text_speek=""
                arekta_text=""
                print("Object Count for Each Class:")
                text_speek += "Object Count for Each Class"


                for class_name, count in class_counts.items():
                    print(f"{class_name}: {count}")
                    arekta_text = str(count) + class_name

                print("\nTotal Object Count:", total_objects)
                text_speek += "Total Object Count"
                text_speek += str(total_objects)

                for class_name, _ in detected_objects:
                    print(f"Detected: {class_name}")
                    text_to_speek= "Detected" + class_name
                    text_speek += text_to_speek

                play_audio(text_speek)


                cv2_imshow(frame)

                #cv2.cv2_imshow(cv2.resize(image_np, (800, 600)))

                cv2.waitKey(1)

            video_camera_capture.release()
            cv2.destroyAllWindows()

        except KeyboardInterrupt:
            pass



if __name__ == '__main__':

    # Logo
    #tprint("Object detection")
    #tprint("by")
    #tprint("paveldat")

    # Loading YOLO scales from files and setting up the network
    net = cv2.dnn.readNetFromDarknet("/content/drive/MyDrive/Thesis_practice/Code 2/Resources/yolov4-tiny.cfg",
                                   "/content/drive/MyDrive/Thesis_practice/Code 2/Resources/yolov4-tiny.weights")
    layer_names = net.getLayerNames()
    out_layers_indexes = net.getUnconnectedOutLayers()
    out_layers = [layer_names[index - 1] for index in out_layers_indexes]

    # Loading from a file of object classes that YOLO can detect
    with open("/content/drive/MyDrive/Thesis_practice/Code 2/Resources/coco.names.txt") as file:
        classes = file.read().split("\n")

    # Determining classes that will be prioritized for search in an image
    # The names are in the file coco.names.txt

    video = input("Give the video path: ")
    #look_for = input("What are we looking for: ").split(',')
    #/content/drive/MyDrive/Thesis_practice/Code 2/Result/input/street.mp4

    # Delete spaces
    look_for = ["item1", "item2", "item3"]

    list_look_for = ["all"]
    for look in look_for:
       list_look_for.append(look.strip())

       classes_to_look_for = list_look_for

    start_video_object_detection(video)


Give the video path: /content/drive/MyDrive/Thesis_practice/Code 2/Result/input/street.mp4


NameError: ignored